In [ ]:
import torch

x = torch.tensor([1., 2.])
y = torch.tensor([4., 5.])
z = torch.tensor([7., 3.])

In [ ]:
x.requires_grad

False

In [ ]:
(x + y).requires_grad

False

In [ ]:
z.requires_grad = True
(x + z).requires_grad

True

In [ ]:
t = torch.tensor([1., 2., 4.]).requires_grad_()
u = torch.tensor([10., 20.]).requires_grad_()
a = t.pow(2).sum() + u.log().sum()
torch.autograd.grad(a, (t, u))

(tensor([2., 4., 8.]), tensor([0.1000, 0.0500]))

In [ ]:
x = torch.tensor([-3., 2., 5.]).requires_grad_()
u = x.pow(3).sum()
u.backward()
x.grad

In [ ]:
w1 = torch.rand(5, 5).requires_grad_()
w2 = torch.rand(5, 5).requires_grad_()
x = torch.empty(5).normal_()

x0 = x
x1 = w1 @ x0
x2 = x0 + w2 @ x1
x3 = w1 @ (x1 + x2)

q = x3.norm()
q.backward()

In [ ]:
(w1.grad, w2.grad)

(tensor([[ 3.4970,  1.3540,  3.0194,  9.6591,  2.4202],
         [ 4.4596,  1.6533,  3.8418, 12.4724,  3.0697],
         [ 3.6690,  1.2448,  3.1469, 10.5039,  2.4992],
         [ 4.5348,  1.7073,  3.9097, 12.6277,  3.1274],
         [ 3.6598,  1.5966,  3.1815,  9.7313,  2.5739]]),
 tensor([[1.0722, 1.7499, 1.4743, 2.3190, 1.7401],
         [1.3782, 2.2492, 1.8951, 2.9807, 2.2366],
         [1.2391, 2.0223, 1.7038, 2.6799, 2.0109],
         [1.3263, 2.1645, 1.8237, 2.8684, 2.1523],
         [0.6271, 1.0235, 0.8623, 1.3563, 1.0177]]))

In [ ]:
x = torch.tensor([1., -2., 3., -4.]).requires_grad_()
a = x.abs()
s = a.sum()

In [ ]:
s

tensor(10., grad_fn=<SumBackward0>)

In [ ]:
s.grad_fn.next_functions

((<AbsBackward0 at 0x7fc303478690>, 0),)

In [ ]:
s.grad_fn.next_functions[0][0].next_functions

((<AccumulateGrad at 0x7fc2fbc2ebd0>, 0),)

In [ ]:
import torch.nn.functional as F

x = torch.zeros(2, 5).requires_grad_()
y = F.relu(x).sum()
y.backward()
x.grad

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [ ]:
import torch.nn as nn

f = nn.Linear(in_features = 10, out_features = 4)
for n, p in f.named_parameters(): print(n, p.size())

weight torch.Size([4, 10])
bias torch.Size([4])


In [ ]:
x = torch.empty(523, 10).normal_()
y = f(x)
y.size()

torch.Size([523, 4])

In [ ]:
f = nn.MSELoss()
x = torch.tensor([[3.]])
y = torch.tensor([[0.]])

f(x, y)

tensor(9.)

In [ ]:
x = torch.tensor([[3., 0., 0., 0.,]])
y = torch.tensor([[0., 0., 0., 0.,]])
f(x, y)

tensor(2.2500)

In [ ]:
import time

def timing(x, w, batch = False, nb = 101):
    t = torch.zeros(nb)

    for u in range(nb):
        t0 = time.perf_counter()
        if batch:
            y = x.mm(w.t())
        else:
            y = torch.empty(x.size(0), w.size(0))
            for k in range(y.size(0)):
                y[k] = w.mv(x[k])
        y.is_cuda and torch.cuda.syncrhnize()
        t[u] = time.perf_counter() - t0
    return t.median().item()

In [ ]:
x = torch.empty(2500, 1000).normal_()
w = torch.empty(1500, 1000).normal_()

In [ ]:
no_batch = timing(x, w, batch = False)
batch = timing(x, w, batch = True)
no_batch, batch

(0.50550776720047, 0.09590647369623184)

In [ ]:
import torch.nn.functional as F

weight = torch.empty(5, 4, 2, 3).normal_()
bias = torch.empty(5).normal_()
input = torch.empty(117, 4, 10, 3).normal_()
output = F.conv2d(input, weight, bias)
output.size()

torch.Size([117, 5, 9, 1])

In [ ]:
import torch.nn as nn

f = nn.Conv2d(in_channels = 4, out_channels = 5, kernel_size = (2, 3))
for n, p in f.named_parameters(): print(n, p.size())

weight torch.Size([5, 4, 2, 3])
bias torch.Size([5])


In [ ]:
x = torch.empty(117, 4, 10, 3).normal_()
y = f(x)
y.size()

torch.Size([117, 5, 9, 1])

In [ ]:
x = torch.empty(1, 2, 2, 6).random_(3)
print(x)
F.max_pool2d(x, (1, 2))

tensor([[[[2., 2., 2., 2., 0., 1.],
          [2., 0., 0., 2., 2., 2.]],

         [[0., 2., 0., 1., 1., 1.],
          [2., 1., 1., 1., 1., 1.]]]])


tensor([[[[2., 2., 1.],
          [2., 2., 2.]],

         [[2., 1., 1.],
          [2., 1., 1.]]]])

In [20]:
from torchvision.datasets import MNIST
mnist = MNIST('./data/mnist/', train = True, download = True)
d = mnist.train_data
d.size()

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:75: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


torch.Size([60000, 28, 28])

In [21]:
x = d.view(d.size(0), 1, d.size(1), d.size(2))
x.size()

torch.Size([60000, 1, 28, 28])

In [23]:
x = x.view(x.size(0), -1)
x.size()

torch.Size([60000, 784])

In [24]:
from torch import nn
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size = 5)
        self.conv2 = nn.Conv2d(32, 64, kernel_size = 5)
        self.fc1 = nn.Linear(256, 200)
        self.fc2 = nn.Linear(200, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), kernel_size=3, stride=3))
        x = F.relu(F.max_pool2d(self.conv2(x), kernel_size=2, stride=2))
        x = x.view(-1, 256)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [27]:
model = Net()
input = torch.empty(12, 1, 28, 28).normal_()
output = model(input)
output

tensor([[ 0.0545, -0.1302, -0.0086,  0.1670, -0.1949,  0.1389,  0.0017,  0.0410,
         -0.1605, -0.0361],
        [ 0.1019, -0.1660, -0.0188,  0.1477, -0.2056,  0.1365, -0.0119,  0.0164,
         -0.1419, -0.0502],
        [ 0.0630, -0.1047,  0.0306,  0.1611, -0.1728,  0.1053, -0.0144,  0.0642,
         -0.1796, -0.0414],
        [ 0.0550, -0.0743,  0.0426,  0.0927, -0.1990,  0.1311,  0.0379,  0.0421,
         -0.2030, -0.0153],
        [ 0.0485, -0.1067,  0.0229,  0.1226, -0.1903,  0.1191,  0.0295,  0.0283,
         -0.1520, -0.0192],
        [ 0.0636, -0.1075,  0.0145,  0.1008, -0.1906,  0.1222,  0.0391,  0.0632,
         -0.1874, -0.0356],
        [ 0.1099, -0.0951,  0.0663,  0.0942, -0.1790,  0.1369, -0.0232,  0.0700,
         -0.1617, -0.0135],
        [ 0.1121, -0.1199,  0.0532,  0.1226, -0.1896,  0.1198, -0.0173,  0.0772,
         -0.2135, -0.0382],
        [ 0.0906, -0.0819,  0.0268,  0.1284, -0.2049,  0.1576, -0.0295,  0.0380,
         -0.1550,  0.0056],
        [ 0.0899, -

In [29]:
model = Net()
for k in model.parameters():
    print(k.size())

torch.Size([32, 1, 5, 5])
torch.Size([32])
torch.Size([64, 32, 5, 5])
torch.Size([64])
torch.Size([200, 256])
torch.Size([200])
torch.Size([10, 200])
torch.Size([10])
